In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import poros

In [2]:
data = pd.read_csv('200_games.csv')
data

,game_id,campeon,frases,resultado
0,1297814575#participant1,Poppy,"Buen uso del barón, Poco daño con Poppy, Torre...",1
1,1297814575#participant2,Diana,"Jungla agresivo, Buen uso del barón, Mucho dañ...",1
2,1297814575#participant3,Zoe,"Buen uso del barón, Destructor de torretas, Bu...",1
3,1297814575#participant4,Kai'Sa,"Buen uso del barón, Destructor de torretas, Bu...",1
4,1297814575#participant5,Pyke,"Buen uso del barón, Ingankeable, KDA alto, Rot...",1
...,...,...,...,...
1995,1298362233#participant6,Volibear,"Buen uso del barón, Destructor de torretas, In...",1
1996,1298362233#participant7,Riven,"Jungla agresivo, Buen uso del barón, KDA alto,...",1
1997,1298362233#participant8,Cassiopeia,"Jugador agresivo, Buen uso del barón, Ingankea...",1
1998,1298362233#participant9,Sivir,"Buen uso del barón, Ingankeable, Buen farmeado...",1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   game_id    2000 non-null   object
 1   campeon    2000 non-null   object
 2   frases     1996 non-null   object
 3   resultado  2000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 62.6+ KB


In [3]:
def preprocesamiento(df):
    df = df.copy()
    
    #Limpiar valores nulos de 'frases'
    missing_frases = df[df['frases'].isna()].index
    df = df.drop(missing_frases, axis=0).reset_index(drop=True)
    
    #Reemplazar nombres de campeones por palabra unica
    df = poros.clean(df)
    
    #Columnas de interes
    y = df['resultado']
    x = df['frases']
    
    #Separacion para entrenamiento
    X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True, random_state=1)
    
    # Learn the vocabulary
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    
    # Find the size of the vocabulary
    vocab_length = len(tokenizer.word_index) + 1
    print("Vocab length:", vocab_length)
    
    # Convert review texts into sequences of integers
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)
    
    
    # Find the maximum sequence length
    max_seq_length = np.max(list(map(lambda x: len(x), X_train)))
    print("Maximum sequence length:", max_seq_length)
    
    
    # Pad the sequences to by uniform length
    X_train = pad_sequences(X_train, maxlen=max_seq_length, padding='post')
    X_test = pad_sequences(X_test, maxlen=max_seq_length, padding='post')
    
    return X_train, X_test, y_train, y_test, vocab_length, max_seq_length, tokenizer

In [4]:
X_train, X_test, y_train, y_test, vocab_length, max_seq_length, tokenizer = preprocesamiento(data)

C:\Course Material Python\scrappingLOL\poros.py:82: FutureWarning: The default value of regex will change from True to False in a future version.
  df['frases'] = df['frases'].str.replace(nombre_campeon, palabra_comun)


Vocab length: 85
Maximum sequence length: 39


In [8]:
X_train.shape

(1596, 39)

## ENTRENAMIENTO

In [9]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

x = tf.keras.layers.Embedding(
    input_dim=vocab_length,
    output_dim=128,
    input_length=max_seq_length
)(inputs)

#Probar accurancy c/s:
x = tf.keras.layers.GRU(256, return_sequences = False, activation = "tanh")(x)


x = tf.keras.layers.Flatten()(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
40/40 [==============================] - 5s 75ms/step - loss: 0.6953 - accuracy: 0.4984 - val_loss: 0.6916 - val_accuracy: 0.5281
Epoch 2/100
40/40 [==============================] - 3s 68ms/step - loss: 0.6940 - accuracy: 0.4828 - val_loss: 0.6927 - val_accuracy: 0.5281
Epoch 3/100
40/40 [==============================] - 3s 68ms/step - loss: 0.6920 - accuracy: 0.5149 - val_loss: 0.6891 - val_accuracy: 0.5094
Epoch 4/100
40/40 [==============================] - 3s 69ms/step - loss: 0.5262 - accuracy: 0.7492 - val_loss: 0.4671 - val_accuracy: 0.7750
Epoch 5/100
40/40 [==============================] - 3s 67ms/step - loss: 0.3788 - accuracy: 0.8699 - val_loss: 0.3808 - val_accuracy: 0.8469
Epoch 6/100
40/40 [==============================] - 3s 74ms/step - loss: 0.3254 - accuracy: 0.8824 - val_loss: 0.3766 - val_accuracy: 0.8625
Epoch 7/100
40/40 [==============================] - 3s 72ms/step - loss: 0.3214 - accuracy: 0.8848 - val_loss: 0.3933 - val_accuracy: 0.8250
Epoch 

## Resultados

In [10]:
model.evaluate(X_test, y_test)

13/13 [==============================] - 1s 16ms/step - loss: 0.3301 - accuracy: 0.8650


[0.33013126254081726, 0.8650000095367432]

In [20]:
# Guardar el modelo
model.save("8650_3301m2.h5")

## Testeo

In [11]:
prediction = model.predict(X_test)

13/13 [==============================] - 0s 11ms/step


In [12]:
prediction[0]

array([0.13496445], dtype=float32)

In [13]:
umbral = 0.5
predicciones = (prediction > umbral).astype(int)
etiquetas = np.squeeze(predicciones)

In [14]:
#partida
p = 8
# Obtener el primer dato de entrada y su correspondiente predicción
primer_dato_entrada = X_test[p]
prediccion_primero = etiquetas[p]

# Imprimir el primer dato de entrada y su predicción
print("Primer dato de entrada:", primer_dato_entrada)
# Reconstruir el texto original
texto_original = tokenizer.sequences_to_texts([primer_dato_entrada])[0]

# Imprimir el texto original y su predicción
print("Texto original:", texto_original)
print("Predicción para el primer dato:", prediccion_primero)

Primer dato de entrada: [ 2 10 11 12 31 16 17 21 18  1  4 15 19 20 32  3 27 30  1 28  3 14  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
Texto original: buen uso del barón mucho daño con campeon destructor de torretas ingankeable kda alto sobreviviente jugador seguro dejó de farmear jugador pasivo
Predicción para el primer dato: 1


## Nuevo input

In [10]:
nuevo_dato = 'Derrota a pesar de ventaja, Baja participación en muertes'

In [5]:
# Cargar el modelo
loaded_model = tf.keras.models.load_model("8650_3301m2.h5")
type(loaded_model)

In [7]:
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 39)]              0         
                                                                 
 embedding (Embedding)       (None, 39, 128)           10880     
                                                                 
 gru (GRU)                   (None, 256)               296448    
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 307,585
Trainable params: 307,585
Non-trainable params: 0
_________________________________________________________________


In [ ]:
new_frase = "Jungla agresivo, Buen farmeador, Torretas ignoradas, Mal duelista, Sin centinelas de control"

In [11]:
new_x,predict = poros.new(nuevo_dato ,loaded_model, max_seq_length,tokenizer)

1/1 [==============================] - 0s 20ms/step


In [47]:
predict[0]

array([0.9006556], dtype=float32)

In [12]:
umbral = 0.5
predicciones_s = (predict > umbral).astype(int)
etiquetas_s = np.squeeze(predicciones_s)

# Obtener el primer dato de entrada y su correspondiente predicción
primer_dato_entrada_s = new_x[0]
prediccion_primero_s = etiquetas_s

# Imprimir el primer dato de entrada y su predicción
print("Primer dato de entrada:", primer_dato_entrada_s)
# Reconstruir el texto original
texto_original_s = tokenizer.sequences_to_texts([primer_dato_entrada_s])[0]

# Imprimir el texto original y su predicción
print("Texto procesado:", texto_original_s)
print("Predicción para el primer dato:", prediccion_primero_s)

Primer dato de entrada: [57 56 58  1 59 35  8  6  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
Texto original: derrota a pesar de ventaja baja participación en muertes
Predicción para el primer dato: 0
